In [1]:
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine

In [ ]:
##For the ETL project, it's a given you're submitting the code. Along with submitting this code, we've given you two options to tell the story that goes along with it: 

#1) A 2 to 3 minute mini-talk (with no visual aids), or
#2) A one-page written report. 

#In either case, we're looking for the following in your story-telling:
#- An outline of the methodology
#- A rundown of the three aspects of ETL
#- Coherence, confidence, and personal style in your communication


In [ ]:
#showing to make sure documentation is correct when using
pd.__version__

In [ ]:
pd.show_versions()

In [2]:
#Extract: your original data sources and how the data was formatted (CSV, JSON, pgAdmin 4, etc).
#load the csv
roster = pd.read_csv('Resources/1996_2016seasons.csv')
roster.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Chris Robinson,VAN,23,195.58,90.71840,Western Kentucky,USA,1996,2,...,4.6,1.7,1.6,-11.4,0.039,0.088,0.155,0.486,0.156,1996-97
1,1,Matt Fish,MIA,27,210.82,106.59412,North Carolina-Wilmington,USA,1992,2,...,0.3,0.8,0.0,-15.1,0.143,0.267,0.265,0.333,0.000,1996-97
2,2,Matt Bullard,HOU,30,208.28,106.59412,Iowa,USA,Undrafted,Undrafted,...,4.5,1.6,0.9,0.9,0.016,0.115,0.151,0.535,0.099,1996-97
3,3,Marty Conlon,BOS,29,210.82,111.13004,Providence,USA,Undrafted,Undrafted,...,7.8,4.4,1.4,-9.0,0.083,0.152,0.167,0.542,0.101,1996-97
4,4,Martin Muursepp,DAL,22,205.74,106.59412,None,USA,1996,1,...,3.7,1.6,0.5,-14.5,0.109,0.118,0.233,0.482,0.114,1996-97


In [3]:
#showing how many columns and rows
roster.shape

(9561, 22)

In [4]:
roster.columns

Index(['Unnamed: 0', 'player_name', 'team_abbreviation', 'age',
       'player_height', 'player_weight', 'college', 'country', 'draft_year',
       'draft_round', 'draft_number', 'gp', 'pts', 'reb', 'ast', 'net_rating',
       'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct', 'season'],
      dtype='object')

In [5]:
#deleting columns we are not adding to database
roster = roster.drop(['net_rating',
       'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct'],axis=1)

In [6]:
roster = roster.rename({'Unnamed: 0':'PlayerID','player_name':'Player Name', 
                        'team_abbreviation':'Team', 'age':'Age',
                        'player_height':'Player Height', 'player_weight':'Player Weight',
                        'college':'College', 'country':"Country", 
                        'draft_year':'Draft Year', 'draft_round':'Draft Round',
                        'draft_number':'Draft Number', 'gp':'Games Played',
                        'pts':'Points', 'reb':'Rebounds', 'ast':'Assists', 
                        'season':'Year'}, axis='columns')

In [7]:
roster.columns

Index(['PlayerID', 'Player Name', 'Team', 'Age', 'Player Height',
       'Player Weight', 'College', 'Country', 'Draft Year', 'Draft Round',
       'Draft Number', 'Games Played', 'Points', 'Rebounds', 'Assists',
       'Year'],
      dtype='object')

In [34]:
#load the csv
box_score = pd.read_csv('Resources/Seasons_Stats.csv')
box_score.tail(5)

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
24686,24686,2017.0,Cody Zeller,PF,24.0,CHO,62.0,58.0,1725.0,16.7,...,0.679,135.0,270.0,405.0,99.0,62.0,58.0,65.0,189.0,639.0
24687,24687,2017.0,Tyler Zeller,C,27.0,BOS,51.0,5.0,525.0,13.0,...,0.564,43.0,81.0,124.0,42.0,7.0,21.0,20.0,61.0,178.0
24688,24688,2017.0,Stephen Zimmerman,C,20.0,ORL,19.0,0.0,108.0,7.3,...,0.600,11.0,24.0,35.0,4.0,2.0,5.0,3.0,17.0,23.0
24689,24689,2017.0,Paul Zipser,SF,22.0,CHI,44.0,18.0,843.0,6.9,...,0.775,15.0,110.0,125.0,36.0,15.0,16.0,40.0,78.0,240.0
24690,24690,2017.0,Ivica Zubac,C,19.0,LAL,38.0,11.0,609.0,17.0,...,0.653,41.0,118.0,159.0,30.0,14.0,33.0,30.0,66.0,284.0


In [35]:
box_score.columns

Index(['Unnamed: 0', 'Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'blanl', 'OWS', 'DWS', 'WS', 'WS/48', 'blank2',
       'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [36]:
box_score = box_score.drop(['Unnamed: 0', 'Age', 'Tm','G', 'GS', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'blanl', 'OWS', 'DWS', 'WS', 'WS/48', 'blank2',
       'OBPM', 'DBPM', 'BPM', 'VORP'],axis=1)

In [37]:
box_score.columns

Index(['Year', 'Player', 'Pos', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [10]:
box_score.Year.unique()

array([1950.,   nan, 1951., 1952., 1953., 1954., 1955., 1956., 1957.,
       1958., 1959., 1960., 1961., 1962., 1963., 1964., 1965., 1966.,
       1967., 1968., 1969., 1970., 1971., 1972., 1973., 1974., 1975.,
       1976., 1977., 1978., 1979., 1980., 1981., 1982., 1983., 1984.,
       1985., 1986., 1987., 1988., 1989., 1990., 1991., 1992., 1993.,
       1994., 1995., 1996., 1997., 1998., 1999., 2000., 2001., 2002.,
       2003., 2004., 2005., 2006., 2007., 2008., 2009., 2010., 2011.,
       2012., 2013., 2014., 2015., 2016., 2017.])

In [46]:
box_score.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [61]:
box_score = box_score[box_score.Year != 1950.]
box_score = box_score[box_score.Year != 1980.]
box_score = box_score[box_score.Year != 1981.]
box_score = box_score[box_score.Year != 1982.]
box_score = box_score[box_score.Year != 1983.]
box_score = box_score[box_score.Year != 1984.]
box_score = box_score[box_score.Year != 1985.]
box_score = box_score[box_score.Year != 1986.]
box_score = box_score[box_score.Year != 1987.]
box_score = box_score[box_score.Year != 1988.]
box_score = box_score[box_score.Year != 1989.]
box_score = box_score[box_score.Year != 1990.]
box_score = box_score[box_score.Year != 1991.]
box_score = box_score[box_score.Year != 1992.]
box_score = box_score[box_score.Year != 1993.]
box_score = box_score[box_score.Year != 1994.]
box_score = box_score[box_score.Year != 1995.]

In [65]:
final_data = pd.merge(roster, box_score[['Player', 'Pos', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS']], 
                      left_on='Player Name',
                      right_on='Player Name', how = left)

NameError: name 'left' is not defined

In [ ]:
final_data.to_csv("final_data.csv", index=False, encoding='utf8')

In [ ]:
#Load: the final database, tables/collections, and why this was chosen

In [ ]:
rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
engine.table_names()

In [ ]:
new_customer_data_df.to_sql(name='final_data.csv', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from Players', con=engine).head()